In [1]:
import polars as pl
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import time

#### Pull Data

In [2]:
df = pl.read_csv(r"C:\Users\Denne\Documents\VSCodeProjects\FinancialStreamline2\BankStreamline\Data\BankDataProd.csv", ignore_errors=True)

df.limit(3)

date,category,description,cost,controllable,monthName,month,day,year,weekDay,cardType,quarter
str,str,str,f64,i64,str,i64,i64,i64,str,str,i64
"""2025-3-31""","""food""","""DC THE BLACK CAT CAFE EL DORAD…",54.61,1,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-31""","""amazon""","""DC eBay O*08-12892-27822 San J…",125.74,1,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-28""","""wmt""","""POS 0328 0135 698593 Walmart.c…",58.47,1,"""Mar""",3,28,2025,"""Fri""","""Debit""",1


#### Edit dataframe

In [3]:
# nf = df.with_columns(category = pl.when(pl.col('description').str.contains('AIRBNB|EXPEDIA')).then(pl.lit('fun')) \
#         .otherwise(pl.col('category')))


# nf = df.with_columns(category = pl.when(pl.col('category') == 'rent').then(pl.lit('mortgage')) \
#         .otherwise(pl.col('category')))


# nf = df.with_columns(category = pl.when(pl.col('category') == 'progressive').then(pl.lit('auto insurance')) \
#         .otherwise(pl.col('category')))

# nf = df.with_columns(category = pl.when((pl.col('category') == 'gas') & (pl.col('description').str.contains('1111'))).then(pl.lit('food')) \
#         .otherwise(pl.col('category')))


nf = df.drop('controllable')

# nf = df.with_columns(quarter = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.quarter())
                                    

# nf = nf.select(['date', 'category', 'description', 'cost', 'controllable', 'monthName', 'month', 'day', 'year', 'weekDay', 'cardType', 'quarter'])
# nf = nf.cast({'controllable':pl.Int16})


# nf['cost'].min()

# nf.filter(pl.col('controllable') == 'error')
# nf.filter((pl.col('category') == 'fun') & (pl.col('description').str.contains('EXPEDIA|AIRBNB')))
# nf.filter(pl.col('description').str.contains('DEPOSIT'))

nf

date,category,description,cost,monthName,month,day,year,weekDay,cardType,quarter
str,str,str,f64,str,i64,i64,i64,str,str,i64
"""2025-3-31""","""food""","""DC THE BLACK CAT CAFE EL DORAD…",54.61,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-31""","""amazon""","""DC eBay O*08-12892-27822 San J…",125.74,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-28""","""wmt""","""POS 0328 0135 698593 Walmart.c…",58.47,"""Mar""",3,28,2025,"""Fri""","""Debit""",1
"""2025-3-27""","""wmt""","""POS 0327 0005 524132 Walmart.c…",6.42,"""Mar""",3,27,2025,"""Thu""","""Debit""",1
"""2025-3-25""","""subscriptions""","""DC Spotify USA 877-7781161 NY …",11.99,"""Mar""",3,25,2025,"""Tue""","""Debit""",1
…,…,…,…,…,…,…,…,…,…,…
"""2025-12-4""","""wmt""","""WM SUPERCENTER #530 EL DO…",21.52,"""Dec""",12,4,2025,"""Thu""","""Credit""",4
"""2025-12-3""","""shopping""","""SALONCENTRIC EL DO…",17.57,"""Dec""",12,3,2025,"""Wed""","""Credit""",4
"""2025-12-3""","""food""","""MURPHY 1111 EL DO…",4.38,"""Dec""",12,3,2025,"""Wed""","""Credit""",4


#### Check for duplicates

In [4]:
nf.filter(nf.is_duplicated())
# len(nf.filter(nf.is_duplicated()))

date,category,description,cost,monthName,month,day,year,weekDay,cardType,quarter
str,str,str,f64,str,i64,i64,i64,str,str,i64
"""2025-3-3""","""misc""","""VENMO PAYMENT 2503…",1.0,"""Mar""",3,3,2025,"""Mon""","""Debit""",1
"""2025-3-3""","""misc""","""VENMO PAYMENT 2503…",1.0,"""Mar""",3,3,2025,"""Mon""","""Debit""",1
"""2025-9-23""","""food""","""MURPHY 1111 EL DO…",1.75,"""Sep""",9,23,2025,"""Tue""","""Credit""",3
"""2025-9-23""","""food""","""MURPHY 1111 EL DO…",1.75,"""Sep""",9,23,2025,"""Tue""","""Credit""",3
"""2025-11-4""","""fun""","""Card purchase UNITED UNITED.C…",577.0,"""Nov""",11,4,2025,"""Tue""","""Debit""",4
"""2025-11-4""","""fun""","""Card purchase UNITED UNITED.C…",577.0,"""Nov""",11,4,2025,"""Tue""","""Debit""",4
"""2025-12-20""","""fun""","""Card purchase UNITED UNITED.C…",35.0,"""Dec""",12,20,2025,"""Sat""","""Debit""",4
"""2025-12-20""","""fun""","""Card purchase UNITED UNITED.C…",35.0,"""Dec""",12,20,2025,"""Sat""","""Debit""",4


#### check nf length
- Mar: 784
- Apr: 853
- Dec 2025: 1322
- Jan 2026: 1485

In [6]:
len(nf)

1485

#### Update file (use w/ caution)

In [ ]:
# path = r'C:\Users\Denne\Documents\VSCodeProjects\FinancialStreamline2\BankStreamline\Data\BankDataProd.csv'

# with open(path, mode="w", encoding="utf-8") as f:
#    nf.write_csv(f, include_header=True)

#### Adhoc looking at data

In [11]:
# list(df.filter(pl.col('category') == 'unknown')['description'].unique())
list(nf['quarter'].unique())

# maybe car words: 'O.REILLY|O.REILLY|MUFFLEX MUFFLER|AUTOZONE|DC TAKE 5|KARL MALONE FORD', 

[1, 2, 3, 4]